In [1]:
from pulsar_qcm.pulsar_qcm import pulsar_qcm
qcm = pulsar_qcm("qcm0", "192.168.0.3", debug=1)
qcm.get_idn()

{'manufacturer': 'Qblox',
 'device': 'Pulsar QCM',
 'serial_number': '',
 'build': {'firmware': {'version': '0.1.0',
   'date': '04/08/2020-07:03:35',
   'hash': '0xD12A1BA',
   'dirty': False},
  'kernel_mod': {'version': '0.1.0',
   'date': '04/08/2020-12:27:50',
   'hash': '0x1128E33B',
   'dirty': True},
  'software': {'version': '0.1.0',
   'date': '04/08/2020-12:27:50',
   'hash': '0x1128E33B',
   'dirty': True}}}

In [2]:
qcm.stop_sequencer()

In [3]:
from quantify.sequencer.types import Schedule, Operation
from quantify.sequencer.gate_library import Rxy
from quantify.sequencer.pulse_library import SquarePulse
from quantify.sequencer.backends.pulsar_backend import pulsar_assembler_backend
from quantify.sequencer.compilation import add_pulse_information_transmon, determine_absolute_timing
from quantify.sequencer.backends.pulsar_backend import PulsarModulations
from quantify.sequencer.resources import QubitResource, CompositeResource, Pulsar_QCM_sequencer, Pulsar_QRM_sequencer

In [4]:
DEVICE_TEST_CFG = {
    "qubits":
    {
        "q1": {"mw_amp180": 0.6, "mw_motzoi": 0.25, "mw_duration": 20e-9,
               "mw_modulation_freq": 200e6, "mw_ef_amp180": 0.87, "mw_ch": "qcm0.s0",
               "ro_ch": "qrm0.s0", "ro_pulse_amp": 0.5, "ro_pulse_modulation_freq": 0,
               "ro_pulse_type": "square", "ro_pulse_duration": 150e-9,
               "ro_acq_delay": 120e-9, "ro_acq_integration_time": 2500e-9,
               "ro_acq_weigth_type": "SSB",
               "init_duration": 250e-6
               }
    },
    "edges": {}
}

In [5]:
qcm.stop_sequencer()

In [6]:
import numpy as np

sched = Schedule('pulsar_params')
q0 = QubitResource('q1')

sched.add(Rxy(45, 0, q0.name))
sched.add(Rxy(180, 0, q0.name))
sched.add(Rxy(90, 0, q0.name))
sched.add(Rxy(180, 90, q0.name))

qcm0_s0 = Pulsar_QCM_sequencer('qcm0.s0', instrument_name='qcm0', seq_idx=0)
sched.add_resources([q0, qcm0_s0])
sched = add_pulse_information_transmon(sched, DEVICE_TEST_CFG)
sched = determine_absolute_timing(sched)
cfg = pulsar_assembler_backend(sched, configure_hardware=True, debug=True)

In [7]:
qcm.arm_sequencer()
qcm.start_sequencer()

In [8]:
qcm.get_assembler_log()

'assembler finished successfully'

In [9]:
qcm.get_sequencer_state(0)

{'status': 'RUNNING', 'flags': []}